In [ ]:
# import transformers
# import torch

# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# pipeline = transformers.pipeline(
#   "text-generation",
#   model="meta-llama/Meta-Llama-3.1-8B-Instruct",
#   model_kwargs={"torch_dtype": torch.bfloat16},
#   device="cuda",
# )

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-674c7de7-1368969e630fdb030017c0b6;d61c2002-aed1-44b1-94fa-9d66fc934ad5)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [8]:
import pandas as pd
import os, glob, sys
import matplotlib.pyplot as plt
# load from csv
dataset_root_dir = "../datasets/poverty" 
experiments_root_dir = "../experiment_results/poverty" 


street_level_image_folder  = f"{dataset_root_dir}/mapillary/imagery"
street_level_metadata_folder  = f"{dataset_root_dir}/mapillary/metadata"
street_level_image_path = glob.glob(os.path.join(street_level_image_folder, "**/*.jpeg"), recursive=True)

metadata_all = pd.read_csv(f"{dataset_root_dir}/metadata_all_duplicates_removed_smaller.csv")
metadata_all['img_path_local'] = metadata_all['img_path'].apply(lambda x: street_level_image_folder + "/" + x)
# reinuemrate the indexes for metadata_all
metadata_all.reset_index(inplace=True)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
from functools import partial
import time
from tqdm import tqdm
from torch.profiler import profile, record_function, ProfilerActivity
import argparse
import sys

sys.path.insert(0, '/home/kaede/yeok/COMP0173/sustainbench/dinov2')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

log_dir = f"{experiments_root_dir}/logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import timm



# Create embedding layer and regression prediction for model
class Model(nn.Module):
    def __init__(self, model):
        super(Model, self).__init__()
        self.feature_model = model
        self.feature_size = 1280
        print(f"Feature size: {self.feature_size}")
        self.embed_and_regression = nn.Sequential(
            nn.Linear(self.feature_size, 128),
            nn.Linear(128, 1)
        )
        
        # Disable gradient for feature model
        for param in self.feature_model.parameters():
            param.requires_grad = False
            
        for param in self.embed_and_regression.parameters():
            param.requires_grad = True
        
    def forward(self, x):
        x = self.feature_model(x)
        x = self.embed_and_regression(x)
        return x
    
model = timm.create_model(
    'mobilenetv4_conv_small_050.e3000_r224_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

model = Model(model)
# Define loss function for regression
criterion = nn.MSELoss()

optimizer = optim.Adam(model.embed_and_regression.parameters(), lr=0.001)

exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


Feature size: 1280


In [ ]:
import numpy as np
# Write a dataloader for the street level images
class StreetLevelDataset(Dataset):
    def __init__(self, metadata, transform=None):
        self.metadata = metadata
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        try:
            row = self.metadata.iloc[idx]
            img = plt.imread(row['img_path_local'])
            img = img.astype(np.float32) / 255.0  # Normalize
            img = img.transpose(2, 0, 1)  # (H, W, C) -> (C, H, W)
            img = torch.tensor(img, dtype=torch.float32).

            if self.transform:
                img = self.transform(img)

            label = torch.tensor(row['labels'], dtype=torch.float32)

            idx = torch.tensor(idx, dtype=torch.int64)
            return idx, img, label

        except Exception as e:
            print(f"Error loading image {idx}: {e}")
            temp_filepath = f"{dataset_root_dir}/mapillary/imagery/AL/AL-2008-5#-00000105/138149814963930.jpeg"
            row = self.metadata.iloc[idx]
            img = plt.imread(temp_filepath)
            img = img.astype(np.float32) / 255.0  # Normalize
            img = img.transpose(2, 0, 1)  # (H, W, C) -> (C, H, W)
            img = torch.tensor(img, dtype=torch.float32)

            if self.transform:
                img = self.transform(img)

            label = torch.tensor(row['labels'], dtype=torch.float32)

            idx = torch.tensor(idx, dtype=torch.int64)
            return idx, img, label


# Instantiate dataset and dataloader
# create dataset
ds_street = StreetLevelDataset(metadata_all, transform=transforms)

# load dataloader
dl_street = DataLoader(
    ds_street,
    batch_size=32,
    shuffle=True,
    num_workers=16,  # Adjust based on the CPU core count
    prefetch_factor=2,
)


In [33]:
import wandb
wandb.init(project='sustainbench', entity='tamagochi')
# wandb.init(project=f"project-nn-search-{N}-{timenow}", config={"system.monitor": False}, reinit = True, id=f"nn_{[f"{h:2.2f}" for h in hyperparams]}")

# Train the model
num_epochs = 5
wandb.config.epochs = num_epochs
best_loss = 0.0
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    

    for phase in ['train', 'val']:
        start_time = time.time()
        if phase == 'train':
            model.feature_model.eval()
            model.embed_and_regression.train()
        else:
            model.feature_model.eval()
            model.embed_and_regression.eval()

        running_loss = 0.0
        
        # iterate over dl
        for idx, inputs, labels in tqdm(dl_street):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            
            # with torch.no_grad():
            features = model(inputs)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model.embed_and_regression(features)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            
        epoch_loss = running_loss / len(dl_street.dataset)
        
        end_time = time.time()
        time_elapsed = end_time - start_time
        print('{} Loss: {:.4f} Time: {:.0f}m {:.0f}s'.format(phase, epoch_loss, time_elapsed // 60, time_elapsed % 60)) # Modify this line
        wandb.log({
            "phase": phase,
            "epoch_loss": epoch_loss,
            "time_elapsed":   time_elapsed,
        })
        
        
        if phase == 'val' and loss > best_loss:
            best_loss = loss
            torch.save(model.state_dict(), os.path.join(log_dir, 'stree_best_model.pth'))

wandb.finish()
print('Training complete')


Epoch 0/4
----------


  0%|          | 0/34835 [00:00<?, ?it/s]

Error loading image 830166: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 651007: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

Error loading image 335459: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error loading image 67507: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 750876: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 885883: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method


Error loading image 967500: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you

  0%|          | 0/34835 [00:00<?, ?it/s]

Error loading image 966974: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error loading image 951917: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 19594: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

Error loading image 460296: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 1001859: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

Error loading image 660141: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start methodError loading image 411281: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/tmp/ipykernel_56698/2960186544.py", line 17, in __getitem__
    img = torch.tensor(img, dtype=torch.float32).to(device)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaede/miniconda3/envs/torch251/lib/python3.12/site-packages/torch/cuda/__init__.py", line 305, in _lazy_init
    raise RuntimeError(
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kaede/miniconda3/envs/torch251/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/kaede/miniconda3/envs/torch251/lib/python3.12/site-packages/torch